In [ ]:
# Cell 1: imports and paths

import os
import pandas as pd

from kalshi_fetcher.kalshi_client import request  # your existing helper

# Notebook lives in notebooks/, data lives in ../data/
DATA_DIR = os.path.join("..", "data")
FEATURES_PATH = os.path.join(DATA_DIR, "features_with_scores.csv")
OUTCOMES_PATH = os.path.join(DATA_DIR, "market_outcomes.csv")

print("DATA_DIR:", DATA_DIR)
print("FEATURES_PATH:", FEATURES_PATH)
print("OUTCOMES_PATH:", OUTCOMES_PATH)
